In [1]:
import json
import pandas as pd

In [2]:
conv_path = '/data/LPJ/ICML25/GraphCoder/graphgpt_dataset/gpt_dataset_construction/more200_gpt4/specific_task/instr_reg/v1/without_head/conversations.json'
graph_path = '/data/LPJ/ICML25/GraphCoder/graphgpt_dataset/gpt_dataset_construction/more200_gpt4/specific_task/instr_reg/v1/without_head/graph.jsonl'

In [3]:
import re

def find_module_string(text):
    # Regex pattern to find the string starting with "module" and ending with ";"
    # It uses non-greedy matching to find the shortest match
    pattern = r"module.*?;"
    
    # Using re.DOTALL to make the dot match all characters including newline
    match = re.search(pattern, text, re.DOTALL)
    
    if match:
        return match.group()  # Returns the matched string
    else:
        return None  # No match found

# Example usage
input_text = """
some initial text
module name="example1" \n   attribu\nte="   value";
some other text
module name="example2" attribute="another value";
"""

# Find the first occurrence
result = find_module_string(input_text)
print("Found substring:", result)

# If you need to find all occurrences instead of just the first one, you can use re.findall
all_matches = re.findall(r"module.*?;", input_text, re.DOTALL)
print("All found substrings:", all_matches)

Found substring: module name="example1" 
   attribu
te="   value";
All found substrings: ['module name="example1" \n   attribu\nte="   value";', 'module name="example2" attribute="another value";']


In [4]:
graph = pd.read_json(graph_path, lines=True)
with open(conv_path, 'r') as f:
    conv = json.load(f)


In [5]:
conv[0]['conversations'][0]['value']

'Given a submodules interconnection graph: \n<graph>\n, \nnodes: [{\'id\': 0, \'content\': \'clk\', \'type\': \'input port\'}, {\'id\': 1, \'content\': \'op_code\', \'type\': \'input port\'}, {\'id\': 2, \'content\': \'func3\', \'type\': \'input port\'}, {\'id\': 3, \'content\': \'func7\', \'type\': \'input port\'}, {\'id\': 4, \'content\': \'alu_op\', \'type\': \'output port\'}, {\'id\': 5, \'content\': \'logic_unit\', \'type\': \'submodule\'}], \nedge_attrs: [], \nconnectivity: [[1, 2, 3, 5], [5, 5, 5, 4]]\n, Design a Verilog module named "alu_control_unit" for a RISC-V ALU that takes an instruction and decodes it to generate control signals for an Arithmetic Logic Unit (ALU). The module should handle a subset of RISC-V instructions that includes add, subtract, and bitwise AND.\n\nModule name:\n    alu_control_unit\n\nInput ports:\n    clk: Clock input for synchronization.\n    op_code [6:0]: 7-bit operation code from the instruction.\n    func3 [2:0]: 3-bit function field from the i

In [6]:
conv[0]['conversations'][1]['value']

"module alu_control_unit (\n    input clk,\n    input [6:0] op_code,\n    input [2:0] func3,\n    input [6:0] func7,\n    output [3:0] alu_op\n);\n\n    wire [3:0] decoded_op;\n\n    decode_logic logic_unit (\n        .op_code(op_code),\n        .func3(func3),\n        .func7(func7),\n        .alu_op(decoded_op)\n    );\n\n    assign alu_op = decoded_op;\n\nendmodule\n\nmodule decode_logic (\n    input [6:0] op_code,\n    input [2:0] func3,\n    input [6:0] func7,\n    output reg [3:0] alu_op\n);\n\n    always @(*) begin\n        case (op_code)\n            7'b0110011: begin // R-type\n                case (func3)\n                    3'b000: alu_op <= (func7 == 7'b0000000) ? 4'b0000 : // ADD\n                                       (func7 == 7'b0100000) ? 4'b0001 : // SUB\n                                       4'b1111; // Undefined\n                    3'b111: alu_op <= 4'b0010; // AND\n                    default: alu_op <= 4'b1111; // Undefined\n                endcase\n            

In [7]:
head = find_module_string(conv[0]['conversations'][1]['value'])
conv[0]['conversations'][0]['value'] + head

'Given a submodules interconnection graph: \n<graph>\n, \nnodes: [{\'id\': 0, \'content\': \'clk\', \'type\': \'input port\'}, {\'id\': 1, \'content\': \'op_code\', \'type\': \'input port\'}, {\'id\': 2, \'content\': \'func3\', \'type\': \'input port\'}, {\'id\': 3, \'content\': \'func7\', \'type\': \'input port\'}, {\'id\': 4, \'content\': \'alu_op\', \'type\': \'output port\'}, {\'id\': 5, \'content\': \'logic_unit\', \'type\': \'submodule\'}], \nedge_attrs: [], \nconnectivity: [[1, 2, 3, 5], [5, 5, 5, 4]]\n, Design a Verilog module named "alu_control_unit" for a RISC-V ALU that takes an instruction and decodes it to generate control signals for an Arithmetic Logic Unit (ALU). The module should handle a subset of RISC-V instructions that includes add, subtract, and bitwise AND.\n\nModule name:\n    alu_control_unit\n\nInput ports:\n    clk: Clock input for synchronization.\n    op_code [6:0]: 7-bit operation code from the instruction.\n    func3 [2:0]: 3-bit function field from the i

In [8]:
for i in range(len(conv)):
    module_head = find_module_string(conv[i]['conversations'][1]['value'])
    conv[i]['conversations'][0]['value'] = conv[i]['conversations'][0]['value'] + '\n' + module_head

In [9]:
# conv[100]['conversations'][0]['value']

In [10]:
new_conv_path = '/data/LPJ/ICML25/GraphCoder/graphgpt_dataset/gpt_dataset_construction/more200_gpt4/specific_task/instr_reg/v1/with_head/conversations.json'



In [11]:
with open(new_conv_path, 'w') as f:
    json.dump(conv, f)

In [12]:
import json
with open(new_conv_path, 'r') as f:
    new_conv = json.load(f)
    


In [13]:
len(new_conv)

480

In [ ]:
new_conv[100]['conversations'][0]['value']

In [32]:
len(graph)

2148